In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.utils import resample
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler



from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics

In [32]:
url = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T00:00:00%27%20and%20%272019-04-12T00:00:00%27"
data = pd.read_json(url)

In [43]:
data.tail()

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
995,ADDRESS,HPD,Department of Housing Preservation and Develop...,4.015110e+09,QUEENS,NaN,NaN,NaN,Elmhurst,2019-04-13T02:13:10.000,...,Unspecified,2019-04-13T02:13:10.000,The complaint you filed is a duplicate of a co...,NaN,Closed,ELMHURST AVENUE,NaN,42197021,1018372.0,211606.0
996,ADDRESS,HPD,Department of Housing Preservation and Develop...,4.015110e+09,QUEENS,NaN,NaN,NaN,Elmhurst,2019-04-13T02:13:10.000,...,Unspecified,2019-04-13T02:13:10.000,The complaint you filed is a duplicate of a co...,NaN,Closed,ELMHURST AVENUE,NaN,42198022,1018372.0,211606.0
997,ADDRESS,HPD,Department of Housing Preservation and Develop...,4.015110e+09,QUEENS,NaN,NaN,NaN,Elmhurst,2019-04-13T02:13:10.000,...,Unspecified,2019-04-13T02:13:10.000,The complaint you filed is a duplicate of a co...,NaN,Closed,ELMHURST AVENUE,NaN,42193360,1018372.0,211606.0
998,ADDRESS,DOT,Department of Transportation,4.103470e+09,QUEENS,NaN,NaN,NaN,SAINT ALBANS,2019-05-02T10:29:27.000,...,Unspecified,2019-05-02T10:29:27.000,The Department of Transportation inspected the...,NaN,Closed,DUNLOP AVENUE,NaN,42200621,1047247.0,195180.0
999,ADDRESS,NYPD,New York City Police Department,3.065660e+09,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2019-04-12T05:03:12.000,...,Unspecified,2019-04-12T05:03:12.000,The Police Department responded and upon arriv...,NaN,Closed,AVENUE M,NaN,42193552,991995.0,163948.0


In [34]:
data.columns

Index(['address_type', 'agency', 'agency_name', 'bbl', 'borough',
       'bridge_highway_direction', 'bridge_highway_name',
       'bridge_highway_segment', 'city', 'closed_date', 'community_board',
       'complaint_type', 'created_date', 'cross_street_1', 'cross_street_2',
       'descriptor', 'due_date', 'facility_type', 'incident_address',
       'incident_zip', 'intersection_street_1', 'intersection_street_2',
       'landmark', 'latitude', 'location', 'location_type', 'longitude',
       'open_data_channel_type', 'park_borough', 'park_facility_name',
       'resolution_action_updated_date', 'resolution_description', 'road_ramp',
       'status', 'street_name', 'taxi_pick_up_location', 'unique_key',
       'x_coordinate_state_plane', 'y_coordinate_state_plane'],
      dtype='object')

In [35]:
data.complaint_type.value_counts()

HEAT/HOT WATER                         146
Illegal Parking                        111
Noise - Residential                     95
Rodent                                  95
Blocked Driveway                        81
Noise                                   77
Street Condition                        40
Noise - Street/Sidewalk                 27
Noise - Vehicle                         19
Noise - Commercial                      18
Sewer                                   15
Sanitation Condition                    14
Water System                            13
General Construction/Plumbing           13
Request Large Bulky Item Collection     13
Derelict Vehicle                        13
UNSANITARY CONDITION                    12
Construction Safety Enforcement         11
Unsanitary Animal Pvt Property          11
PAINT/PLASTER                           10
Dirty Conditions                         9
Food Poisoning                           9
Street Light Condition                   8
PLUMBING   

In [36]:
data.borough.value_counts()

BROOKLYN         307
MANHATTAN        237
QUEENS           225
BRONX            180
STATEN ISLAND     47
Unspecified        4
Name: borough, dtype: int64

In [19]:
test = data.groupby(['borough', 'complaint_type'])

In [37]:
data['complaint_type'][data['borough']=='BROOKLYN'].value_counts()

Illegal Parking                        45
Rodent                                 39
HEAT/HOT WATER                         33
Noise - Residential                    29
Blocked Driveway                       24
Street Condition                       13
Noise                                  13
Noise - Commercial                      8
General Construction/Plumbing           7
Water System                            6
PAINT/PLASTER                           6
Sanitation Condition                    5
Sewer                                   4
Noise - Helicopter                      4
Derelict Vehicle                        4
Unsanitary Animal Pvt Property          4
Derelict Vehicles                       4
UNSANITARY CONDITION                    4
FLOORING/STAIRS                         4
Noise - Street/Sidewalk                 4
Noise - Vehicle                         3
PLUMBING                                3
Air Quality                             3
Dirty Conditions                  

In [38]:
data['complaint_type'][data['borough']=='QUEENS'].value_counts()


Blocked Driveway                       35
Illegal Parking                        34
Rodent                                 16
Noise - Residential                    15
HEAT/HOT WATER                         14
Noise                                  14
Request Large Bulky Item Collection    11
Street Condition                        8
Sanitation Condition                    6
Derelict Vehicle                        6
Noise - Vehicle                         6
Noise - Street/Sidewalk                 5
Building/Use                            4
Food Poisoning                          4
Noise - Commercial                      4
General Construction/Plumbing           4
Derelict Vehicles                       3
Street Sign - Missing                   3
Graffiti                                3
ELECTRIC                                2
Water System                            2
Elevator                                2
Air Quality                             2
Unsanitary Animal Pvt Property    

In [39]:
data['complaint_type'][data['borough']=='MANHATTAN'].value_counts()


Noise                                  46
HEAT/HOT WATER                         41
Noise - Residential                    22
Rodent                                 18
Noise - Street/Sidewalk                15
Construction Safety Enforcement        11
Illegal Parking                         8
Street Condition                        7
Noise - Vehicle                         7
Noise - Commercial                      6
Sewer                                   5
PLUMBING                                4
Dirty Conditions                        3
UNSANITARY CONDITION                    3
PAINT/PLASTER                           3
Food Poisoning                          3
Homeless Person Assistance              3
For Hire Vehicle Complaint              2
Water System                            2
DOOR/WINDOW                             2
Unsanitary Animal Pvt Property          2
WATER LEAK                              2
Blocked Driveway                        2
GENERAL                           

In [42]:
data['complaint_type'][data['borough']=='BRONX'].value_counts()


HEAT/HOT WATER                    58
Noise - Residential               27
Rodent                            20
Blocked Driveway                  15
Illegal Parking                   14
Street Condition                   6
UNSANITARY CONDITION               4
Sewer                              3
Noise                              3
Elevator                           3
Noise - Street/Sidewalk            3
Sanitation Condition               3
Street Light Condition             2
Dirty Conditions                   2
Derelict Vehicle                   2
Unsanitary Animal Pvt Property     2
PLUMBING                           1
Standing Water                     1
GENERAL                            1
Animal Abuse                       1
Noise - Vehicle                    1
Sidewalk Condition                 1
Hazardous Materials                1
Food Poisoning                     1
ELECTRIC                           1
Traffic                            1
New Tree Request                   1
W

In [41]:
data['complaint_type'][data['borough']=='STATEN ISLAND'].value_counts()


Illegal Parking                      10
Street Condition                      6
Blocked Driveway                      5
Water System                          3
Standing Water                        2
Sewer                                 2
Rodent                                2
WATER LEAK                            2
Noise - Vehicle                       2
Noise - Residential                   2
Electronics Waste Appointment         2
Street Light Condition                2
Litter Basket / Request               1
Unsanitary Animal Pvt Property        1
Traffic Signal Condition              1
UNSANITARY CONDITION                  1
Noise                                 1
Derelict Vehicle                      1
Missed Collection (All Materials)     1
Name: complaint_type, dtype: int64

In [45]:
data.created_date.tail()

995    2019-04-11T07:59:25.000
996    2019-04-11T07:59:26.000
997    2019-04-11T07:59:28.000
998    2019-04-11T07:59:34.000
999    2019-04-11T07:59:37.000
Name: created_date, dtype: object

In [54]:
url1 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T08:00:00%27%20and%20%272019-04-12T00:00:00%27"
data1 = pd.read_json(url1)

In [53]:
data.created_date.tail()

995    2019-04-11T07:59:25.000
996    2019-04-11T07:59:26.000
997    2019-04-11T07:59:28.000
998    2019-04-11T07:59:34.000
999    2019-04-11T07:59:37.000
Name: created_date, dtype: object

In [57]:
data1.created_date.tail()

995    2019-04-11T09:49:00.000
996    2019-04-11T09:49:00.000
997    2019-04-11T09:49:00.000
998    2019-04-11T09:49:00.000
999    2019-04-11T09:49:03.000
Name: created_date, dtype: object

In [56]:
data1.complaint_type.value_counts()

HEAT/HOT WATER                             104
Street Light Condition                      91
Illegal Parking                             73
Missed Collection (All Materials)           51
Request Large Bulky Item Collection         46
Street Condition                            46
UNSANITARY CONDITION                        34
Derelict Vehicles                           33
Blocked Driveway                            30
General Construction/Plumbing               26
Dirty Conditions                            26
Sanitation Condition                        22
Noise - Residential                         22
PAINT/PLASTER                               19
Water System                                19
Graffiti                                    17
Noise                                       14
Sidewalk Condition                          14
WATER LEAK                                  14
Derelict Vehicle                            13
ELECTRIC                                    13
DOOR/WINDOW  

In [58]:
url2 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T09:49:04%27%20and%20%272019-04-12T00:00:00%27"
data2 = pd.read_json(url2)

In [61]:
data2.created_date.tail()

995    2019-04-11T11:38:27.000
996    2019-04-11T11:38:36.000
997    2019-04-11T11:38:36.000
998    2019-04-11T11:38:36.000
999    2019-04-11T11:38:36.000
Name: created_date, dtype: object

In [65]:
url3 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T11:38:37%27%20and%20%272019-04-12T00:00:00%27"
data3 = pd.read_json(url3)

In [67]:
data3.created_date.tail()

995    2019-04-11T13:35:16.000
996    2019-04-11T13:35:24.000
997    2019-04-11T13:35:24.000
998    2019-04-11T13:36:00.000
999    2019-04-11T13:36:06.000
Name: created_date, dtype: object

In [69]:
url4 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T13:36:06%27%20and%20%272019-04-12T00:00:00%27"
data4 = pd.read_json(url4)
data4.created_date.tail()

995    2019-04-11T15:25:05.000
996    2019-04-11T15:25:09.000
997    2019-04-11T15:25:14.000
998    2019-04-11T15:25:17.000
999    2019-04-11T15:25:22.000
Name: created_date, dtype: object

In [70]:
url5 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T15:25:22%27%20and%20%272019-04-12T00:00:00%27"
data5 = pd.read_json(url5)
data5.created_date.tail()

995    2019-04-11T17:29:44.000
996    2019-04-11T17:29:44.000
997    2019-04-11T17:29:45.000
998    2019-04-11T17:29:46.000
999    2019-04-11T17:30:00.000
Name: created_date, dtype: object

In [71]:
url6 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T17:30:00%27%20and%20%272019-04-12T00:00:00%27"
data6 = pd.read_json(url6)
data6.created_date.tail()

995    2019-04-11T20:29:55.000
996    2019-04-11T20:30:00.000
997    2019-04-11T20:30:11.000
998    2019-04-11T20:30:33.000
999    2019-04-11T20:30:35.000
Name: created_date, dtype: object

In [75]:
data6.created_date[999]

'2019-04-11T20:30:35.000'

In [80]:
url7 = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T20:30:35%27%20and%20%272019-04-11T23:59:48%27"
data7 = pd.read_json(url7)
data7.created_date.tail(10)

984    2019-04-11T23:57:41.000
985    2019-04-11T23:57:52.000
986    2019-04-11T23:58:00.000
987    2019-04-11T23:58:07.000
988    2019-04-11T23:58:12.000
989    2019-04-11T23:58:52.000
990    2019-04-11T23:59:00.000
991    2019-04-11T23:59:00.000
992    2019-04-11T23:59:00.000
993    2019-04-11T23:59:47.000
Name: created_date, dtype: object

In [ ]:
df.to_csv('NewSavedView.csv', index=False)